# Design oligo sequences of STRs used for lentiMPRA

In [1]:
# imports
import argparse
import hashlib
import numpy as np
import pandas as pd
import os
import pyfaidx
import random
import subprocess
import sys

from trtools.utils import utils

In [22]:
# Global variables for oligo construction. These cannot change
# R2 primer-(var + genomic context)-minP-spacer-filler

# FIVE_PRIME_ADAPT = 'AGGACCGGATCAACT'

R2_PRIMER = 'GTGCTCTTCCGATCT'
GLOBAL_FILLER_SEQ = 'TAACCAGGCGTGTTAGCTGCTGTGCTGTCCTACGAGTAAACAGTAGAGTCCGTGGGCACGCGAGCACGGTGAGTCGACTCTGGCCTCATCACCATTTAGTTTGCGCAAGCGCTCTTTTTATAGGACCTGTCTTACATCCCTCATTAACGGAATCGATTACCGGCTAGCGTTGAAATGGAGAAACCGGCTTGCAGTCGAAA'
MIN_PROMOTER = 'CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCCGGTACTGT'
SPACER = 'GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG'
PROBE_LEN = 300

MIN_STR_LEN = 0
MAX_STR_LEN = 105

MAX_ALLELE_LEN = 5
MIN_FILLER_LEN = 0
REQUIRED_ELTS_LEN = len(R2_PRIMER) + len(MIN_PROMOTER) + len(SPACER)

MAX_VREG = PROBE_LEN - (REQUIRED_ELTS_LEN)

In [23]:
def create_probes (path_genome, path_bed_file, outdir):
    """
    creates oligo library to be used for MPRA experiment to study STRs
    parameter:
        1. path_genome
            - path to the genome of the organism of interest
        2. path_bed_file
            - path to the bed file 
                > must contain chr, start_str, end_str, rpt_unit
    output:
    """

    # create an empty list to be converted to a dataframe later
    array_probes_list = []

    # load in fasta files using pyfaidx and bed files using pandas
    genome = pyfaidx.Fasta(path_genome)
    colnames=['chr', 'start_str', 'end_str', 'rpt_unit', 
              'name', 'description', 'type']
    bed_df = pd.read_csv(path_bed_file, names=colnames)

    bed_df['len_str'] = bed_df['end_str'] - bed_df['start_str']

    name_list = bed_df['name']
    chromosome = bed_df['chr']
    start = bed_df['start_str']
    end = bed_df['end_str']
    rptUnit = bed_df['rpt_unit']

    # figure out what is the maximum context length that can be generated

    # add in a column called max_str_size which is the longest str region that
    # will be generated (5 * lenght of rpt unit) + length of the str region
    bed_df['max_str_size'] = (MAX_ALLELE_LEN * bed_df['rpt_unit']).str.len() + \
        bed_df['len_str']
    
    # max_rpt_unit_size is the maximum of the max_str_size column
    max_rpt_unit_size = np.max(bed_df['max_str_size'])
    
    # chooses the smaller length between the maximum length of str and maximum
    # repeat unit length
    max_bp_len = np.min([MAX_STR_LEN, max_rpt_unit_size])

    # gives the maximum amount of context that can be generated; all oligos
    # follow this maximum
    ### could also generate maximum for each oligo (unsure if we want that...)
    max_context_size = PROBE_LEN - REQUIRED_ELTS_LEN - MIN_FILLER_LEN - \
        max_bp_len
    
    copy_num_variation = ['m5', 'p5']

    for (chrom, str_start, str_end, rpt, name) in \
        zip(chromosome, start, end, rptUnit, name_list):

        max_rpt_unit_size = (MAX_ALLELE_LEN*len(rpt)+(str_end-str_start))
        max_bp_len = np.min([MAX_STR_LEN, max_rpt_unit_size])
        max_context_size = PROBE_LEN - REQUIRED_ELTS_LEN - MIN_FILLER_LEN - \
            max_bp_len

        left_flank = str(genome[chrom][str_start-int(np.ceil(max_context_size/2)):str_start]).upper()
        str_refseq = str(genome[chrom][str_start:str_end]).upper()
        right_flank = str(genome[chrom][str_end:str_end+int(np.floor(max_context_size/2))]).upper()

        ref_seq = name+','+chrom+','+str(str_start)+','+str(str_end)+','+rpt+\
                    ',ref,'+left_flank+','+str_refseq+','+right_flank
    

        for copy_num_var in copy_num_variation:
            if copy_num_var == "p5":
                str_p5seq = str_refseq + (5*rpt)
                p5_seq = name+','+chrom+','+str(str_start)+','+str(str_end)+','+rpt+\
                    ',p5,'+left_flank+','+str_p5seq+','+right_flank
                array_probes_list.append(p5_seq)
            if copy_num_var == "m5":
                str_m5seq = str_refseq[0:-1*(len(rpt) * 5)]
                if (len(str_m5seq) < len(rpt)) & ((len(str_m5seq)) != 0) & ((len(str_m5seq) % len(rpt)) != 0):
                    str_m5seq = rpt
                m5_seq = name+','+chrom+','+str(str_start)+','+str(str_end)+','+rpt+\
                    ',m5,'+left_flank+','+str_m5seq+','+right_flank
                array_probes_list.append(m5_seq)

        array_probes_list.append(ref_seq)

    split_strings = [string.split(',') for string in array_probes_list]
    column_headers = ['name', 'chr', 'str start', 'str end', 'motif', 'allele', 
                      'left', 'str', 'right']
    context_df = pd.DataFrame(split_strings, columns=column_headers)

    context_df = context_df[context_df['str'].str.len() <= 100]

    context_df['probeseq'] = context_df['left'] + context_df['str'] + context_df['right']
    context_df['STR_id'] = context_df['name']+"_"+context_df['allele']

    context_df = context_df.drop(labels='name', axis=1)
    
    vreg_len = (context_df['probeseq']).str.len()
    vreg = context_df['probeseq']

    filler_len = []
    oligo_list = []
    size_check = []    

    for i in vreg_len:
        filler_len.append(PROBE_LEN-i-REQUIRED_ELTS_LEN)

    for j, seq in zip(filler_len, vreg):
        assert(j >= MIN_FILLER_LEN)
        filler_seq = GLOBAL_FILLER_SEQ[0:j]
        size_check.append(R2_PRIMER + seq + MIN_PROMOTER + 
                          SPACER + filler_seq)
        oligo_list.append(R2_PRIMER + '\t' + seq + '\t' + MIN_PROMOTER + '\t' + 
                          SPACER + '\t' + filler_seq)
        
    for line in size_check:
        assert(len(line) == PROBE_LEN)
    
    split_strings = [string.split('\t') for string in oligo_list]
    column_headers = ['R2 primer', 'probeseq', 'minP', 'spacer', 
                    'filler']
    oligo_df = pd.DataFrame(split_strings, columns=column_headers)

    final_df = pd.merge(left=context_df, right=oligo_df)
    final_df['final_oligo'] = final_df['R2 primer'] + final_df['probeseq'] + \
        final_df['minP'] + final_df['spacer'] + final_df['filler']

    final_df = final_df.drop_duplicates(subset='final_oligo')

    final_df.to_csv(outdir+'probeseq.tsv')
    return(final_df)

In [24]:
oligos = create_probes(path_genome='/Users/user/Desktop/MPRA/lentiSTR design/hg38.fa', 
                       path_bed_file='/Users/user/Downloads/lentiSTRs-v2_with_hSTR_changed_startsite_collapsed.csv',
                       outdir='/Users/user/Desktop/')

In [25]:
oligos

,chr,str start,str end,motif,allele,left,str,right,probeseq,STR_id,R2 primer,minP,spacer,filler,final_oligo
0,chr1,983460,983479,GTTT,m5,GAGCTCTTCATTGAACTCTTGTACTGGGGAATCTTCTCCCTCTCTG...,,AGACGGAGTTTCACTCTTGTCGCCCAGGCTGATGTGCAATGGCAAA...,GAGCTCTTCATTGAACTCTTGTACTGGGGAATCTTCTCCCTCTCTG...,Human_STR_283_m5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,TAACCAGGCGTGTTAGCTGCTGTGCTGTCCTACGAGTAA,GTGCTCTTCCGATCTGAGCTCTTCATTGAACTCTTGTACTGGGGAA...
1,chr1,983460,983479,GTTT,p5,GAGCTCTTCATTGAACTCTTGTACTGGGGAATCTTCTCCCTCTCTG...,TTGCTTGTTTGTTTGTTTGGTTTGTTTGTTTGTTTGTTT,AGACGGAGTTTCACTCTTGTCGCCCAGGCTGATGTGCAATGGCAAA...,GAGCTCTTCATTGAACTCTTGTACTGGGGAATCTTCTCCCTCTCTG...,Human_STR_283_p5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,,GTGCTCTTCCGATCTGAGCTCTTCATTGAACTCTTGTACTGGGGAA...
2,chr1,983460,983479,GTTT,ref,GAGCTCTTCATTGAACTCTTGTACTGGGGAATCTTCTCCCTCTCTG...,TTGCTTGTTTGTTTGTTTG,AGACGGAGTTTCACTCTTGTCGCCCAGGCTGATGTGCAATGGCAAA...,GAGCTCTTCATTGAACTCTTGTACTGGGGAATCTTCTCCCTCTCTG...,Human_STR_283_ref,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,TAACCAGGCGTGTTAGCTGC,GTGCTCTTCCGATCTGAGCTCTTCATTGAACTCTTGTACTGGGGAA...
3,chr1,1273794,1273819,CCG,m5,TCGAGGCCGGAGCAGGGTGCGGAAAAGCGGGGCCCGCGCCCGAAAC...,CCGCCGCAGC,TCAGCCTCCAAGATGGCCGCTCGCGGGGCGGAACCAGCCTAGCTTC...,TCGAGGCCGGAGCAGGGTGCGGAAAAGCGGGGCCCGCGCCCGAAAC...,Human_STR_456_m5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,TAACCAGGCGTGTTAGCTGCTGTGCTGTCC,GTGCTCTTCCGATCTTCGAGGCCGGAGCAGGGTGCGGAAAAGCGGG...
4,chr1,1273794,1273819,CCG,p5,TCGAGGCCGGAGCAGGGTGCGGAAAAGCGGGGCCCGCGCCCGAAAC...,CCGCCGCAGCGCCGCCGCCGCCGCCCCGCCGCCGCCGCCG,TCAGCCTCCAAGATGGCCGCTCGCGGGGCGGAACCAGCCTAGCTTC...,TCGAGGCCGGAGCAGGGTGCGGAAAAGCGGGGCCCGCGCCCGAAAC...,Human_STR_456_p5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,,GTGCTCTTCCGATCTTCGAGGCCGGAGCAGGGTGCGGAAAAGCGGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,chr9,136866478,136866494,CCTGG,p5,CTCGGGGCCATTCGGCGTTCGGCTCGTCTGAGGGACGTCGTGGCCG...,CCTGGCCTGGCCTGGCCCTGGCCTGGCCTGGCCTGGCCTGG,GAGGCGCGGTAGCCGCTGGGGCTGCAGGTCGGGAGGACCCCGGAGC...,CTCGGGGCCATTCGGCGTTCGGCTCGTCTGAGGGACGTCGTGGCCG...,Human_STR_1525622_p5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,,GTGCTCTTCCGATCTCTCGGGGCCATTCGGCGTTCGGCTCGTCTGA...
1175,chr9,136866478,136866494,CCTGG,ref,CTCGGGGCCATTCGGCGTTCGGCTCGTCTGAGGGACGTCGTGGCCG...,CCTGGCCTGGCCTGGC,GAGGCGCGGTAGCCGCTGGGGCTGCAGGTCGGGAGGACCCCGGAGC...,CTCGGGGCCATTCGGCGTTCGGCTCGTCTGAGGGACGTCGTGGCCG...,Human_STR_1525622_ref,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,TAACCAGGCGTGTTAGCTGCTGTGC,GTGCTCTTCCGATCTCTCGGGGCCATTCGGCGTTCGGCTCGTCTGA...
1176,chr9,137087116,137087133,CCG,m5,TCGGTTCCTCTCAGTCGGACTTCCTGACGCCGCCAGTGGGCGGGGC...,CCG,TCATCGGGACTTCATCTCGGTGACGCTGAGCTTTGGCGAGAACTAT...,TCGGTTCCTCTCAGTCGGACTTCCTGACGCCGCCAGTGGGCGGGGC...,Human_STR_1525766_m5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,TAACCAGGCGTGTTAGCTGCTGTGCTGTC,GTGCTCTTCCGATCTTCGGTTCCTCTCAGTCGGACTTCCTGACGCC...
1177,chr9,137087116,137087133,CCG,p5,TCGGTTCCTCTCAGTCGGACTTCCTGACGCCGCCAGTGGGCGGGGC...,CCGCCGCCGCCGCCGCCCCGCCGCCGCCGCCG,TCATCGGGACTTCATCTCGGTGACGCTGAGCTTTGGCGAGAACTAT...,TCGGTTCCTCTCAGTCGGACTTCCTGACGCCGCCAGTGGGCGGGGC...,Human_STR_1525766_p5,GTGCTCTTCCGATCT,CACTAGAGGGTATATAATGGAAGCTCGACTTCCAGCTTGGCAATCC...,GCAAAGTGAACACATCGCTAAGCGAAAGCTAAG,,GTGCTCTTCCGATCTTCGGTTCCTCTCAGTCGGACTTCCTGACGCC...


In [124]:
def generate_control_oligos (path_control_file, outdir):
    """
    creates oligo library of controls used in 'A Systematic Evaluation...' paper
    parameter:
        1. path_control_file
            - 1st column: haplo name
            - 2nd column: pos/neg control
            - 3rd column: raw sequence
    output:
        1. array probes file
    """

    control_probes_list = []

    control_df = pd.read_csv(path_control_file)
    seq_list = control_df['seq']
    seq_length = (control_df['seq']).str.len()

    filler_len = []
    size_check = []

    for length in seq_length:
        filler_len.append(PROBE_LEN-length-REQUIRED_ELTS_LEN)
    for j, seq in zip(filler_len, seq_list):
        assert(j >= MIN_FILLER_LEN)
        filler_seq = GLOBAL_FILLER_SEQ[0:j]
        size_check.append(R2_PRIMER+seq+MIN_PROMOTER+SPACER+filler_seq)
        control_probes_list.append(R2_PRIMER+'\t'+seq+'\t'+MIN_PROMOTER+'\t'+ \
                                   SPACER+'\t'+filler_seq)
        
    for line in size_check:
        assert(len(line) == PROBE_LEN)

    
    split_strings = [string.split('\t') for string in control_probes_list]

    column_headers = ['R2 primer', 'probeseq', 'minP', 'spacer', 
                    'filler']
    oligo_df = pd.DataFrame(split_strings, columns=column_headers)
    oligo_df['final_oligo'] = oligo_df['R2 primer'] + oligo_df['probeseq'] + \
        oligo_df['minP'] + oligo_df['spacer'] + oligo_df['filler']

    oligo_df = oligo_df.drop_duplicates(subset='final_oligo')

    oligo_df.to_csv(outdir+'pos_neg_controls.tsv')  
    return(oligo_df)  
